<a href="https://colab.research.google.com/github/musicjae/cs231n/blob/master/probability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install d2l==0.14.1
!pip install tensorflow


https://d2l.ai/chapter_preliminaries/probability.html# 참고

## 2.6 Probability  
  
  &nbsp;&nbsp;&nbsp;&nbsp;머신 러닝은 거의 예측하는 일을 수행한다. 가령 우리는 임상적 역사를 고려할 때 다음 해에 심장 마비로 고통받을 환자의 확률을 예측하고자 할 수 있다. 확률은 우리에게 확실성의 수준에 대해 추리하는 형식적인 방법을 제공해준다. 가령, 만약 우리가 어떤 이미지가 고양이를 묘사한다는 것을 완전히 확신한다면, 우리는 라벨 y가'cat'일 확률 - P(y = 'cat') - 은 1이라고 말한다. 만약 우리가 y = 'cat' 이거나 y = 'dog'임을 암시하는 증거가 없다면, 우리는 이 두 가지 가능성에 대해 동일한 확률을 부여하며 P(y='cat')=P(y='dog')=0.5 라고 표현한다.  
 


   
##### **2.6.1 기본적인 확률론** 
  
  {1,2, ..., 6}의 값을 관찰해보자. 각각의 산출값의 확률을 조사해보자. 하나의 자연스러운 접근은 이 값의 개별 셈을 하는 것이고, 리스트 내의 전체 수의 개수에서 그것을 나누는 것이다. 이것은 주어진 사건의 확률값의 추정을 제공해준다.<Br>&nbsp;&nbsp;&nbsp;&nbsp; 대수의 법칙The law of large numbers(LLN)는 우리에게 가령 동전 던지기 경우에 동전을 던지는 횟수가 커질수록 이 추정값은 점점 더 참값에 가까워진다는 것을 시사한다. (데이터가 많을수록 추정이 정확해진다)이런 확률을 사용하려면 아래를 보라:
    

In [ ]:
%matplotlib inline
from d2l import tensorflow as d2l
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

##### 통계학에서 확률 분포로부터 사례를 이끌어내는 과정을 **샘플링**이라고 부른다. 또한, 여러 개의 값을 가질 수 있는 독립 확률 변수에 대한확률 분포를 **다항분포**라고 부르는데, 이것을 가지고 아래에서 샘플링을 수행해보자. i th 인덱스의 값은 이 샘플링이 산출하는 i에 상응하는 것이 몇 번 나타났는지에 대한 것이다. 

In [ ]:
fair_probs = tf.ones(6) / 6
tfp.distributions.Multinomial(1, fair_probs).sample()


In [ ]:
tfp.distributions.Multinomial(10, fair_probs).sample()

##### 샘플링 방법에 대해 간략히 알았으니 이제 1000 개의 주사위를 굴렸다고 있는 시나리오를 떠올려보자. 각각의 1000 개의 주사위를 굴린 뒤, 1 - 6 까지 각각의 수가 얼마나 많이 나왔는지를 보자:

In [ ]:
counts = tfp.distributions.Multinomial(1000, fair_probs).sample()
counts / 1000

##### 위 결과는 1 - 6 이 나타난 빈도에 대한 확률이 모두 거의 $\frac{1}{6}$에 가깝다는 것을 확인했다. <br><br> 우리는 또한 어떻게 이 확률들이 시간이 지남에 따라 참인 확률로 수렴하는지를 시각화활 수 있다. 각각의 그룹이 10 개의 샘플을 뽑는 실험을 500 개의 그룹이 수행했다고 해보자.

In [ ]:
counts = tfp.distributions.Multinomial(10, fair_probs).sample(500)
cum_counts = tf.cumsum(counts, axis=0)
estimates = cum_counts / tf.reduce_sum(cum_counts, axis=1, keepdims=True)

d2l.set_figsize((6, 4.5))
for i in range(6):
    d2l.plt.plot(estimates[:, i].numpy(),
                 label=("P(die=" + str(i + 1) + ")"))
d2l.plt.axhline(y=0.167, color='black', linestyle='dashed')
d2l.plt.gca().set_xlabel('Groups of experiments')
d2l.plt.gca().set_ylabel('Estimated probability')
d2l.plt.legend();

### 2.6.1.1 확률론의 공리  
  
#####  &nbsp;&nbsp;&nbsp;&nbsp;형식적으로, 확률은 어떤 집합에서 어떤 값을 사상하는 함수라고 간주될 수 있다. 주어진 샘플 공간 S 내의 어떤 사건 A의 확률은 P(A)라고 표기하고, 이는 다음의 성질들을 만족한다:

- For any event  A , its probability is never negative, i.e.,  𝑃(S)≥0 ;

- Probability of the entire sample space is  1 , i.e.,  𝑃(S)=1 ;

- For any countable sequence of events  A1,A2,…  that are mutually exclusive ( Ai ∩ Aj = $\emptyset$ for all 𝑖≠𝑗 ), the probability that any happens is equal to the sum of their individual probabilities  
  
  이 공리 체계 때문에, 우리는 임의성randomness에 대한 어떤 철학적 논쟁도 피할 수 있다. 대신에, 우리는 수학 언어를 가지고 엄격하게 추리해낼 수 있다. 가령, 사건 A1을 전체의 샘플 공간이라고 하고, ∀ i > 1 에 대하여, $A_i=\emptyset$이라고 해보자. 그러면 우리는, $P(\emptyset)=0$, 즉 불가능한 사건의 확률은 0임을 증명해낼 수 있다.<br><br>

### 2.6.1.2 랜덤 변수  <br>
  
##### 랜덤 변수 X의 분포를 P(X)라고 하자. 이 분포는 X에 어떤 값도 들어갈 수 있는 확률을 말한다. P(a)는 랜덤 변수가 특정 값 a를 취할 때의 확률을 지칭한다.<br><br>
### 2.6.2 랜덤 변수 다루기<br><br>
- Joint 확률 (A & B)
-조건적 확률 
-베이즈 정리
-Marginalization
-독립성
-적용
-기댓값과 분산


